In [2]:
def letterbox(
    img,
    new_shape=(640, 640),
    color=(0, 0, 0),
    scaleup=True,
):
    """
    指定されたサイズに画像をリサイズを行う。

    Parameters
    --------------
    img : (ndarray) 画像のndarray配列
    new_shape : (tuple) リサイズしたい画像サイズ
    color : (tuple:(B, G, R)) パディングの色
    scaleup : (Bool) スケールアップを行う場合はTrue

    Return
    -----------
    padded_img : (ndarray) リサイズされた画像のndarray配列
    ratio : (tuple リサイズした比率
    (dw, dh) : (int:(横、高さ)) パディングした数値(横、高さ)
    """
    
    # アスペクト比を保った画像のリサイズ
    ori_shape = img.shape[:2] # [H, W, C] -> [H, W]
    r = min(new_shape[0] / ori_shape[0], new_shape[1] / ori_shape[1])
    
    # スケールアップなしの場合
    if not scaleup:
        r = min(r, 1.0)
    
    # アスペクト比を保ったまま、リスケール
    ratio = (r, r)
    new_unpad = (round(ori_shape[1] * r), round(ori_shape[0] * r)) # cv.resizeのため[W, H]にする
    
    # パディングする領域を算出
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    dw, dh = dw / 2, dh / 2

    # dw, dfが.5の時の対応 -> この微調整をしないとsession.run()時のinputのshapeが合わなくなる
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    
    # 画像のリサイズ
    if new_shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
        
    padded_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return padded_img, ratio, (dw, dh)

In [3]:
def xywh2xyxy(x):
    """
    [x中心、y中心、bboxの幅, bboxの高さ] -> [bboxの左上のx、bboxの左上のy, bboxの右下のx, bboxの右下のy]
    """
    y = x.clone()
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # bboxの左上のx
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # bboxの左上のy
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bboxの右下のx
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bboxの右下のy
    return y

In [4]:
def non_max_suppression(
    prediction, 
    conf_thres=0.25,
    iou_thres=0.45,
    agnostic=False,
    labels=(),
    max_det=300,
    nm=0,
):
    """
    
    YOLOの予測結果をNMS処理する。

    Parameters
    --------------
    prediction : (tensor) テンソル化したyoloが予測した結果
    conf_thres : (float) バウンディングボックス(bbox)の信頼度スコアの閾値
    iou_thres : (float) IOUの閾値
    agnostic : (Bool) 重なった異なるクラスのbboxを同一のbboxにするか
    labels : (tuple or list-like) 画像内のラベルの情報
    max_det : (int) 最大のbbox数
    nm : (int) マスク数
    
    Return
    -----------
    output : (tensor) NMS処理後のbbox情報を含むテンソル、[x1, y1, x2, y2, クラス確率, cls_id]のカラムに変換されている
                      x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す
                      
    """
    batch_size = prediction.shape[0] # バッチサイズ
    num_class = prediction.shape[2] - nm - 5 # クラス数
    bool_cnf = prediction[..., 4] > conf_thres # bbox毎に信頼度スコアがconf_thresより大きいかのbool
    
    max_wh = 7680  # 最大のbboxの幅、高さ
    max_nms = 30000  # torchvision.ops.nms()のための最大のbbox数
    time_limit = 0.5 + 0.05 * batch_size  # タイムリミット(s)
    
    start = time.time()
    mi = 5 + num_class
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * batch_size
    
    
    for idx, x in enumerate(prediction):
    
        # conf_thresより大きい信頼度スコアを持つbboxsを抽出
        x = x[bool_cnf[idx]]
    
        # labelsを持っていた時の処理
        if labels and len(labels[idx]):
                lb = labels[idx]
                v = torch.zeros((len(lb), num_class + nm + 5), device=x.device)
                v[:, :4] = lb[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(lb)), lb[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)
            
    
        # フィルター後のbboxがない場合
        if not x.shape[0]:
            continue
            
        x[:, 5:] *= x[:, 4:5] # 信頼度スコア*各cls確率
        box = xywh2xyxy(x[:, :4]) # [x1, y1, x2, y2]:bboxの左上と右下の座標に変換
        mask = x[:, mi:] # maskがなければ[]
    
        cls_prob, cls_id = x[:, 5:mi].max(1, keepdim=True) # 各bboxで確率の高いclsの値とcls_idを返す
    
        # [x1, y1, x2, y2, clsの確率, cls_id]のカラムの順に結合
        # clsの確率(本来のprob*信頼度スコア)がconf_thresより高いもののみ抽出
        x = torch.cat((box, cls_prob, cls_id.float(), mask), axis=1)[cls_prob.view(-1) > conf_thres]
    
        
        n = x.shape[0]
        if not n:
            continue
    
        # cls確率をキーにして降順に並び替え、且つ max_nmsを超えないようにする
        x = x[x[:, 4].argsort(descending=True)[:max_nms]] 
    
        # 異なるクラスのbboxを区別して評価するため、agnosticでクラス固有のbboxを作成
        c = x[:, 5:6] * (0 if agnostic else max_wh)
        boxes, scores = x[:, :4] + c, x[:, 4]
    
        # nms
        selected_idx = torchvision.ops.nms(boxes, scores, iou_thres)
        selected_idx = selected_idx[:max_det]

        output[idx] = x[selected_idx]
        finish = time.time()
        if (finish - start) > time_limit:
            break
            
    return output


In [5]:
#==============
# 画像の前処理
#==============

np_img = cv2.imread(input_file)

# 画像のリサイズ
img = letterbox(np_img, [640, 640])[0]

# (H,W,C) -> (C, H, W)
# BGR -> RGB
img = img.transpose((2, 0, 1))[::-1]
img = np.ascontiguousarray(img)

# テンソル化 -> 正規化
img = torch.from_numpy(img)
img = img.half() if fp16 else img.float()
scaled_img = img / 255
scaled_img = scaled_img.unsqueeze(0)


# ===============
# 推論 
# ===============

scaled_img = scaled_img.cpu().numpy()
output_name = session.get_outputs()[0].name
input_name = session.get_inputs()[0].name

# yのshape:[データ数、bbox数, メタ情報]
# メタ情報:[x中心, y中心, bboxの幅, bboxの高さ, 信頼スコア、 押印の確率, 未押印の確率]
y = session.run([output_name], {input_name: scaled_img})[0]
pred = torch.from_numpy(y) # テンソル化

# NMS処理
pred = non_max_suppression(pred, conf_thres, iou_thres, agnostic_nms, max_det=max_det)



NameError: name 'cv2' is not defined

In [ ]:

# ===============
# 出力処理 
# ==============
pred_cls_list = pred[0][:, -1] # 全bboxのクラス識別結果
no_seal_cls_id = list(eval(meta['names']))[1] # 未押印のcls_id


# 未押印の識別があったら、画像出力の分岐
if no_seal_cls_id in pred_cls_list:

    # --- 未押印あり -> bboxつきの画像配列を渡す処理 -------------
    for det in pred:
        process_img = np_img.copy() # np化したオリジナルimg
        if len(det):
    
            # bboxの情報をオリジナル画像用に調整
            det[:, :4] = scale_boxes(scaled_img.shape[2:], det[:, :4], process_img.shape).round()
    
            # bbox毎にイテレート
            for *xyxy, prob, cls_id in reversed(det):
                cls_id = int(cls_id)
    
                # bboxの色 (cv2に合わせるため、BGR)、YOLOv5に合わせている
                # class_id:[0, 1]の2クラス用 -> いずれclassにしたら、インスタンス変数にしたい
                color = [(56, 56, 255), (151, 157, 255)][cls_id]
                label = names if hide_conf else f"{names[cls_id]} {prob:.2f}"
    
                # 最後にPILで処理されて,ChannelがRGBに変わっているので注意!!
                process_img = box_label(img = process_img, box = xyxy, label = label, color = color)
    
            # POSTのResponseのため、RGBに変換後、(C, H, W)に転置する。
            output_img = cv2.cvtColor(process_img, cv2.COLOR_BGR2RGB)
            output_img = np.transpose(output_img, (2, 0, 1))

    # 未押印cls_idとbbox付き画像のndarray配列を返す
    print('return 1, output_img')

else:
    
    # --- 未押印なし -> ファイル名とコメントを返す -------------
    # filename = os.path.basename(input_file)
    # comment = '未押印なし'

    # # Responseするoutput [filename, comment, 押印のcls_id]
    # output = [filename, comment, 0]
    # print('return output')
    # print('return only seal')

    # --- 未押印なし -> cls_idとNoneを返す -------------

    print("return 0, None")
